In [1]:
import pandas as pd
import json
import pickle
import os
import itertools
from ast import literal_eval
import numpy as np
import sys
from datetime import datetime
from IPython.display import clear_output, display
import subprocess
from datetime import datetime
pd.options.display.float_format = '{:.0f}'.format

In [2]:
def LoadJsonFile(filename): 
    with open(filename, 'r') as f:
        DicConfig = json.load(f)
    return DicConfig


def GlobalDicDeplier(OneDic):
    for k,v in OneDic.items():
        exec('globals()[k] = v')
    return None

In [3]:
DicConfig = LoadJsonFile(os.path.join(os.getcwd(),"config.json"))
GlobalDicDeplier(DicConfig)
sys.path.append(Root)
from fun import *

print("Load Config variables")

Load Config variables


In [4]:
path = os.path.join(Root,FolderProject,"RefFam.pkl")
RefFam = LoadPickleOrInit(path)

path = os.path.join(Root,FolderProject,"RefRT.pkl")
RefRT = LoadPickleOrInit(path)

tweetsdf = RefFam.copy()[RefFam.status=="ok"]
rtdf = RefRT.copy()[RefRT.status=="ok"]

In [5]:
datefirst = rtdf.TWEETUNIXEPOCH.min()
datelast = rtdf.TWEETUNIXEPOCH.max()

In [9]:
WindowList = ExtractWindows(datefirst,datelast,WindowSize,StepSize)

In [27]:
marka,markb = WindowList[0]

In [28]:
rtdf = rtdf[(rtdf.TWEETUNIXEPOCH>=marka) & (rtdf.TWEETUNIXEPOCH<=markb)].reset_index(drop=True)

In [39]:
TweetsFrequency = rtdf.groupby("AUTHORTWEETID").size().reset_index().rename(columns = {0:"f"})

In [41]:
TfidfData = TweetsFrequency.merge(tweetsdf)

In [50]:
print(TfidfData.AUTHORTWEETCONTENT[125])

Harcelez la faf pour qu'il jette cette conasse raciste et negrophile https://t.co/F7bLeVVICr


In [46]:
TfidfData

,AUTHORTWEETID,f,AUTHORID,AUTHORTWEETCONTENT,AUTHORTWEETUNIXEPOCH,BassineID,status
0,1221432958046949120,3,627543473,"Va te faire foutre, sincèrement.\nL’Angola est...",1580050884,438903,ok
1,1221434397167099904,7,893519851028959232,.@gaspardgantzer Merci de rappeler à vos candi...,1580051228,438903,ok
2,1221434490595225600,22,529712945,Il est plus que nécessaire que les parents bla...,1580051250,438903,ok
3,1221435456824520704,5,2968249786,Merci @WonderSo_ pour cette essentielle mise ...,1580051480,438903,ok
4,1221436189842071552,9,1025507290907123456,Honnêtement ça me tient vraiment à cœur de don...,1580051655,438903,ok
...,...,...,...,...,...,...,...
124,1221494575619026944,3,957702865874771968,Je crois que beaucoup de SJW me détestent en g...,1580065575,438907,ok
125,1221494763251289856,7,1213253155384782848,Harcelez la faf pour qu'il jette cette conasse...,1580065620,438907,ok
126,1221495052507197440,10,1044928850,"Le génocide par substitution, crime contre l’h...",1580065689,438907,ok
127,1221495236658114304,1,2333842430,C'est mieux elle reste raciste dans son coin elle,1580065733,438907,ok


In [8]:
def ExtractWindows(datefirst,datelast,WindowSize,StepSize):
    
    start = datefirst
    L = []
    
    while(True):
        end = start+WindowSize
        if end>datelast:
            break
        L.append((start,end))
        start = end - (WindowSize - StepSize)

    return L